In [1]:
import os

if os.path.basename(os.getcwd()) != 'HUST-NLP-Medical-MultiDocument-Summarization-':
    %cd ../../

e:\pyenv\GTCC\KPG-RL\HUST-NLP-Medical-MultiDocument-Summarization-


In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from transformers import LEDForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoTokenizer
import torch
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from tqdm.notebook import tqdm

In [3]:
RANDOM_SEED = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
PATH = './model/centrum'
tokenizer = AutoTokenizer.from_pretrained(PATH)

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(PATH)

In [6]:
DOC_SEP_ = "<doc-sep>"
docsep_token_id = tokenizer.convert_tokens_to_ids(DOC_SEP_)

In [7]:
class Test_Medical_Dataset(Dataset):
    def __init__(self,tokenizer:AutoTokenizer,test_data,test_label):
        self.data = test_data
        self.label = test_label
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self,id):
        sentence = self.data.at[id,'Abstracts']
        target = self.label.at[id,'Target']
        review_id = self.data.at[id,"ReviewID"]
        encoding = self.tokenizer(sentence, return_tensors='pt', padding='max_length', truncation=True, max_length=4096)
        global_attention_mask = [[1 if y in [tokenizer.cls_token_id, docsep_token_id] else 0 for y in x]
                                                 for x in encoding['input_ids']]
        return {
            'input_ids': encoding['input_ids'].squeeze(0), # Squeeze to remove the extra dimension
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': target,
            'global_attention_mask': torch.tensor(global_attention_mask).squeeze(0),
            'abstracts': sentence,
            'review_id': review_id
        }

In [8]:
cochrane_dev_input = pd.read_csv("datasets/mslr_data/cochrane/dev-inputs.csv")
cochrane_dev_input['Abstract'].fillna("",inplace = True)
cochrane_dev_input = cochrane_dev_input.groupby('ReviewID').apply(lambda group:"".join([f"{row['Title']}{DOC_SEP_}{row['Abstract']}{DOC_SEP_}" for index, row in group.iterrows()])).reset_index(name="Abstracts")
cochrane_dev_input.sort_values(by='ReviewID', inplace=True)
cochrane_dev_input.reset_index(drop=True, inplace=True)
cochrane_dev_target = pd.read_csv("datasets/mslr_data/cochrane/dev-targets.csv")
cochrane_dev_target.sort_values(by='ReviewID', inplace=True)
cochrane_dev_target.reset_index(drop=True, inplace=True)

test_dataset = Test_Medical_Dataset(tokenizer,cochrane_dev_input,cochrane_dev_target)

In [9]:
res = pd.DataFrame()
res["Abstracts"] = None
res["Label"] = None
res["Generate"] = None
res["ReviewID"] = None

In [10]:
model.to(device)

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50266, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50266, 768, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
      (layers): ModuleList(
        (0-5): 6 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
          )
     

In [11]:
max_iter = -1

In [12]:
for i,k in enumerate(test_dataset):
    print(i)
    gen = model.generate(input_ids=k['input_ids'].unsqueeze(0).to(device),attention_mask=k['attention_mask'].unsqueeze(0).to(device),global_attention_mask=k['global_attention_mask'].unsqueeze(0).to(device),max_length=1024, num_beams=4, early_stopping=True)
    generated_sentence = tokenizer.decode(gen[0], skip_special_tokens=True)
    res.at[i,"Abstracts"] = k['abstracts']
    res.at[i,"Label"] = k['labels']
    res.at[i,"Generate"] = generated_sentence
    res.at[i,"ReviewID"] = k['review_id']
    if i==max_iter-1:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyError: 470

In [13]:
res

,Abstracts,Label,Generate,ReviewID
0,Fetal lung maturation: the combined use of cor...,"Prenatal TRH, in addition to corticosteroids, ...",There is insufficient evidence to support the ...,CD000019
1,[Ante-partum administration of preventive trea...,The risk of Rhesus D alloimmunisation during o...,There is insufficient evidence to support or r...,CD000020
2,[Anticonvulsant treatment in severe preeclamps...,Magnesium sulphate more than halves the risk o...,Magnesium sulphate does not reduce the risk of...,CD000025
3,Long-term hospitalization and beta-mimetic the...,"Larger, well-designed studies are needed to ev...",There is insufficient evidence to support or r...,CD000036
4,Treatment of acute asthma: salbutamol via jet ...,Metered-dose inhalers with spacer produced out...,There is insufficient evidence to determine wh...,CD000052
...,...,...,...,...
465,The year-long effect of HIV-positive test resu...,This systematic review's findings show that in...,There is limited evidence on the effectiveness...,CD010119
466,Reduction of painful vaso-occlusive crisis of ...,"Based on the results of one study, evidence is...",There is currently insufficient evidence to su...,CD010155
467,Tranexamic acid in hip fracture surgery: a ran...,There is evidence that tranexamic acid reduces...,There is insufficient evidence to support or r...,CD010245
468,Does the peritoneum need to be closed at lapar...,There is no evidence for any short-term or lon...,There is insufficient evidence to support or r...,CD010424


In [15]:
res.to_csv("res.csv")